In [1]:
import pandas as pd

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [14]:
import dask.dataframe as dd
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pyarrow.compute as pc
import pyarrow.feather as ft

In [2]:
cite = pd.read_csv('data/Merged_UK.csv')

In [3]:
cite.head()

,Citing_app_nbr,Cited_App_nbr,IPC_Citing,reg_code_citing,prio_year_citing,IPC_Cited,reg_code_Cited,prio_year_cited
0,EP19790300237,EP19780300148,A61K031/43,UKI23,1978.0,A61K031/41,UKI23,1977
1,EP19790300237,EP19780300148,A61K031/43,UKI23,1978.0,A61K031/41,UKJ13,1977
2,EP19790301143,EP19780100823,A61K031/415,UKH23,1978.0,A61K,UKJ42,1977
3,EP19790301145,EP19780100823,A61K031/415,UKH33,1978.0,A61K,UKJ42,1977
4,EP19790301144,EP19780100823,A61K031/415,UKH23,1978.0,A61K,UKJ42,1977


In [5]:
inv = pd.read_csv('data/inv_pharma_wIPC.csv')
inv.head()

,app_nbr,appln_id,person_id,inv_name,address,reg_code,ctry_code,prio_year,app_year,IPC
0,EP19780100395,16427449.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1977,1978,A61K038/21
1,EP19780100395,16427449.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1977,1978,C07K014/555
2,EP19780100395,16427449.0,2214748.0,"Johnston, Michael Denis","16 Altyre Way,Beckenham Kent",UKJ42,GB,1977,1978,C12P021/00
3,EP19780100463,16427512.0,2214885.0,"Wilkinson, Samuel","12 Bevington Road,Beckenham, Kent",UKJ42,GB,1977,1978,A61K038/00
4,EP19780100463,16427512.0,2214885.0,"Wilkinson, Samuel","12 Bevington Road,Beckenham, Kent",UKJ42,GB,1977,1978,A61P001/12


In [47]:
len(inv)

207766

In [10]:
citing_app_nbr_set = set(cite['Citing_app_nbr'])
app_nbr_set = set(inv['app_nbr'])
diff_set = citing_app_nbr_set.difference(app_nbr_set)
new_df = cite[cite['Citing_app_nbr'].isin(diff_set)]
new_df.head()

,Citing_app_nbr,Cited_App_nbr,IPC_Citing,reg_code_citing,prio_year_citing,IPC_Cited,reg_code_Cited,prio_year_cited
4957,EP19970929378,EP19810901804,C07K016/18,UKH12,1996.0,B05D005/08,UKI23,1980
4958,EP19970929378,EP19810901804,C07K016/18,UKK43,1996.0,B05D005/08,UKI23,1980
4963,EP19990956245,EP19840902466,C12N001/38,UKF21,1998.0,B05B015/00,UKH33,1984
4964,EP19990932045,EP19860904252,A61K008/34,UKJ23,1998.0,A61G005/04,UKG22,1985
4965,EP19990932045,EP19860904252,A61K008/34,UKI23,1998.0,A61G005/04,UKG22,1985


In [12]:
row = inv.loc[inv['app_nbr'] == 'EP19990932045']

print(row.head())

Empty DataFrame
Columns: [app_nbr, appln_id, person_id, inv_name, address, reg_code, ctry_code, prio_year, app_year, IPC]
Index: []


In [42]:
new_df = new_df.drop(['Cited_App_nbr','IPC_Cited','reg_code_Cited','prio_year_cited'], axis = 1)
new_df = new_df.rename(columns={"Citing_app_nbr": "app_nbr","IPC_Citing": "IPC","reg_code_citing":"reg_code","prio_year_citing":"prio_year"})
new_df.head()

,app_nbr,IPC,reg_code,prio_year
4957,EP19970929378,C07K016/18,UKH12,1996.0
4958,EP19970929378,C07K016/18,UKK43,1996.0
4963,EP19990956245,C12N001/38,UKF21,1998.0
4964,EP19990932045,A61K008/34,UKJ23,1998.0
4965,EP19990932045,A61K008/34,UKI23,1998.0


In [50]:
len(new_df)

347

In [43]:
new_df['prio_year'] = new_df['prio_year'].astype(str)

In [48]:
inv_new = inv.append(new_df, ignore_index=True)

/tmp/ipykernel_3758/4252897809.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  inv_new = inv.append(new_df, ignore_index=True)


In [49]:
len(inv_new)

208113

In [53]:
mis = inv_new['reg_code'].isnull().sum()
mis

0

In [54]:
inv_new.to_csv('data/inv_pharma_wIPC.csv')

In [15]:
ipc = dd.read_parquet('data_process/ipc_pharma.parquet')

In [16]:
ipc.head()

,appln_id,prio_year,app_year,IPC
8,2,1991,1992,C07K014/00
9,2,1991,1992,C07K016/00
10,2,1991,1992,C07K016/18
11,2,1991,1992,C07K016/24
12,2,1991,1992,C07K016/26


In [23]:
ipc = ipc.compute()

In [34]:
ipc['prio_year'] = ipc['prio_year'].astype(int).astype(str)
ipc_2017 = ipc[ipc['prio_year'] == '2017']
ipc_2016 = ipc[ipc['prio_year'] == '2016']

In [40]:
print(len(ipc_2017))
print(len(ipc_2016))

156
2217


In [26]:
print(ipc['prio_year'].unique())

[1991 2002 2000 2003 2005 2004 1995 2008 1978 1985 1986 1990 2007 1999
 1998 1988 1994 1996 2001 2006 1989 1992 1993 1997 1984 1983 1987 1981
 1982 1977 1979 1980 2009 2010 2011 2012 2013 2014 2015 2016 2017]
